In [1]:
from selenium import webdriver
import selenium
from time import sleep
import html_to_json
import pandas as pd
import numpy as np
from datetime import datetime as dt
from datetime import timedelta

from itertools import chain

In [2]:
OBSDT = dt.today()  #  ВРЕМЯ ИЗМЕРЕНИЯ
OBSDT = OBSDT - timedelta(seconds=OBSDT.second, microseconds=OBSDT.microsecond)
COUNTRY = 'АНГЛИЯ'
TOURNAMENT = 'ПРЕМЬЕР-ЛИГА'

In [3]:
MONTH_DICT = {"янв": 1,  "фев": 2,  "мар": 3, 
              "апр": 4,  "май": 5,  "июн": 6, 
              "июл": 7,  "авг": 8,  "сен": 9, 
              "окт": 10, "ноя": 11, "дек": 12}

In [4]:
# URL = 'https://www.parimatch.ru/ru/football/premier-league-7f5506e872d14928adf0613efa509494/prematch'
URL = 'https://www.parimatch.ru/ru/football/super-league-b9ca937509c1440aaaacb4fd7c593b80/prematch'
driver = webdriver.Chrome()
driver.get(URL)
sleep(10)
content = driver.page_source
driver.close()

In [5]:
content[:100]

'<html lang="ru" style="--screen-height:674px;"><head><script src="//z.cdn.adpool.bet/au?a=19&amp;&am'

In [6]:
content_json = html_to_json.convert(content)

In [7]:
# content_json

In [8]:
len(content_json)

1

In [9]:
content_json.keys()

dict_keys(['html'])

In [10]:
web_data = content_json['html'][0]['body'][0]
# web_data = content_json

In [11]:
print(len(web_data))
print(web_data.keys())
print(web_data)

7
dict_keys(['_attributes', 'noscript', 'svg', 'script', 'div', 'style', 'iframe'])
{'_attributes': {'data-brand': 'RU'}, 'noscript': [{'iframe': [{'_attributes': {'src': 'https://www.googletagmanager.com/ns.html?id=GTM-5C7JJXD>m_auth=>m_preview=>m_cookies_win=x', 'height': '0', 'width': '0', 'style': 'display:none;visibility:hidden', 'id': 'tag-manager'}}]}, {'div': [{'img': [{'_attributes': {'src': 'https://mc.yandex.ru/watch/46187409', 'style': 'position:absolute; left:-9999px;', 'alt': ''}}]}]}], 'svg': [{'_attributes': {'xmlns': 'http://www.w3.org/2000/svg', 'xmlns:xlink': 'http://www.w3.org/1999/xlink', 'style': 'position: absolute; width: 0; height: 0', 'aria-hidden': 'true', 'id': '__SVG_SPRITE_NODE__'}, 'symbol': [{'_attributes': {'xmlns': 'http://www.w3.org/2000/svg', 'viewbox': '0 0 24 24', 'id': 'AI_Audio'}, 'path': [{'_attributes': {'fill-rule': 'evenodd', 'd': 'M21 3H3c-1.1 0-2 .9-2 2v14c0 1.1.9 2 2 2h18c1.1 0 2-.9 2-2V5c0-1.1-.9-2-2-2zm0 16H3V5h18v14zm-10.5-7C9.117 12 8 

In [ ]:
one_set_data = web_data['div'][0]['div'][1]['div'][0]['div'][1]['div'][2]['div'][0]['div'][0]['div']

In [ ]:
print(len(one_set_data))
try:
    print(one_set_data.keys())
except:
    print('no keys')
one_set_data

In [ ]:
len(one_set_data)

In [ ]:
type(one_set_data)

In [ ]:
def cleaning_games(raw_game_list):
    clean_game_list = []
    for elem in raw_game_list:
        try:
            clean_game_list.append(elem['a'][0]['div'])
        except:
            pass
    return clean_game_list

In [ ]:
clean_game_list = []
for elem in one_set_data:
    try:
        clean_game_list.append(elem['a'][0]['div'])
    except:
        pass

In [ ]:
game_data = one_set_data[10]['a'][0]['div']

In [ ]:
print(len(game_data))
try:
    print(game_data.keys())
except:
    print('no keys')
game_data

# ДАННЫЕ ОБ ИГРЕ

In [ ]:
cats = game_data[0]['div']

In [ ]:
print(len(cats))
try:
    print(cats.keys())
except:
    print('no keys')
print(cats)

In [ ]:
team1 =  cats[0]['div'][1]['span'][0]['_value']
team2 =  cats[2]['div'][1]['span'][0]['_value']
date =  cats[1]['div'][1]['span'][0]['_value']
gtime =  cats[1]['div'][1]['span'][1]['_value']

In [ ]:
print(team1, team2, date, gtime)

In [ ]:
if date.lower() == 'завтра':
    gdt = dt(dt.now().year, (dt.now()+ timedelta(days = 1)).month, (dt.now()+ timedelta(days = 1)).day,
                int(gtime[:2]), int(gtime[-2:]), 0) 
elif date.lower() == 'сегодня':
    gdt = dt(dt.now().year, dt.now().month, dt.now().day,
             int(gtime[:2]), int(gtime[-2:]), 0)
else:
    gdt = dt(dt.now().year, MONTH_DICT[date[-3:]], int(date[:2]), int(gtime[:2]), int(gtime[-2:]), 0)

In [ ]:
gdt

In [ ]:
pd.DataFrame([gdt])

# КЭФЫ

In [ ]:
nums = game_data[1]['div']

In [ ]:
coef = {}
p1 = nums[0]['div'][0]['div'][0]['div']
x = nums[0]['div'][0]['div'][1]['div']
p2 = nums[0]['div'][0]['div'][2]['div']

coef[p1[0]['_value']] = float(p1[1]['div'][0]['span'][0]['_value']) #П1
coef[x[0]['_value']] = float(x[1]['div'][0]['span'][0]['_value']) #X
coef[p2[0]['_value']] = float(p2[1]['div'][0]['span'][0]['_value']) #П2
coef

In [ ]:
game = [gdt, OBSDT, team1, team2, COUNTRY, TOURNAMENT, coef['П1'], coef['Х'], coef['П2']]

In [ ]:
columns = ['GAME_DT', 'OBSDT', 'HT', 'GT', 'COUNTRY', 'TOURNAMENT', 'HW_COEF', 'DR_COEF', 'GW_COEF']
aa = pd.DataFrame([game], columns = columns)

In [ ]:
aa

In [ ]:
df = pd.DataFrame(columns = ['GAME_DT', 'OBSDT', 'HT', 'GT', 'COUNTRY', 'TOURNAMENT', 'HW_COEF', 'DR_COEF', 'GW_COEF'])    
df

In [ ]:
df.loc[1] = game
df

In [ ]:
def game_data_extraction(game_data, OBSDT, COUNTRY, TOURNAMENT):
    cats = game_data[0]['div']
    nums = game_data[1]['div']
    
    team1 =  cats[0]['div'][1]['span'][0]['_value']
    team2 =  cats[2]['div'][1]['span'][0]['_value']
    date =  cats[1]['div'][1]['span'][0]['_value']
    time =  cats[1]['div'][1]['span'][1]['_value']
    
    
    p1 = nums[0]['div'][0]['div'][0]['div']
    x = nums[0]['div'][0]['div'][1]['div']
    p2 = nums[0]['div'][0]['div'][2]['div']
    coef = {}
    coef[p1[0]['_value']] = float(p1[1]['div'][0]['span'][0]['_value']) #П1
    coef[x[0]['_value']] = float(x[1]['div'][0]['span'][0]['_value']) #X
    coef[p2[0]['_value']] = float(p2[1]['div'][0]['span'][0]['_value']) #П2
    
    return [gdt, OBSDT, team1, team2, COUNTRY, TOURNAMENT, coef['П1'], coef['Х'], coef['П2']]

In [ ]:
game_data_extraction(game_data, OBSDT, COUNTRY, TOURNAMENT)

In [ ]:
game

In [ ]:
def tour_data_extraction(tour_data, OBSDT, COUNTRY, TOURNAMENT):
    df = pd.DataFrame(columns = ['GAME_DT', 'OBSDT', 'HT', 'GT', 'COUNTRY', 'TOURNAMENT', 'HW_COEF', 'DR_COEF', 'GW_COEF'])    
    i = 0    
#     for game_data in [elem['a'][0]['div'] for elem in tour_data]:
    for i in range(len(tour_data)): 
        try:
            game_data = tour_data[i]['a'][0]['div']
#             print(i)
            df.loc[i] = game_data_extraction(game_data, OBSDT, COUNTRY, TOURNAMENT)
            i += 1
#             print(df)
        except:
            pass
    return df

In [ ]:
tour_data_extraction(tour_data = one_set_data, OBSDT = OBSDT, COUNTRY = COUNTRY, TOURNAMENT = TOURNAMENT)

In [ ]:
# def page_data_extraction(page_data, OBSDT, COUNTRY, TOURNAMENT):
#     df_page = pd.DataFrame(columns = ['GAME_DT', 'OBSDT', 'HT', 'GT', 'COUNTRY', 'TOURNAMENT', 'HW_COEF', 'DR_COEF', 'GW_COEF']) 
#     for tour_data in [elem['div'] for elem in page_data[1:]]:
#         try:
#             df_add = tour_data_extraction(tour_data, OBSDT, COUNTRY, TOURNAMENT)
#             df_page = pd.concat([df_page, df_add])
#         except:
#             pass
#     df_page.reset_index(drop = True, inplace = True)
#     return df_page

In [ ]:
# page_data_extraction(page_data = data2, OBSDT = OBSDT, COUNTRY = COUNTRY, TOURNAMENT = TOURNAMENT)

## Создание списка всех игр

In [ ]:
web_data = content_json['html'][0]['body'][0]

In [ ]:
one_set_data = web_data['div'][0]['div'][1]['div'][0]['div'][1]['div'][2]['div'][0]['div'][0]['div']

In [ ]:
game_data = one_set_data[19]['a'][0]['div']

In [ ]:
class sport_page:
    def __init__(self, COUNTRY, TOURNAMENT):
        self.COUNTRY = COUNTRY
        self.TOURNAMENT = TOURNAMENT
        self.MONTH_DICT = {"янв": 1,  "фев": 2,  "мар": 3, 
                          "апр": 4,  "май": 5,  "июн": 6, 
                          "июл": 7,  "авг": 8,  "сен": 9, 
                          "окт": 10, "ноя": 11, "дек": 12}
        
    def game_data_extraction(gmdt, OBSDT, COUNTRY, TOURNAMENT, MONTH_DICT):

        team1 =  gmdt[0]['div'][0]['div'][1]['span'][0]['_value']
        team2 =  gmdt[0]['div'][2]['div'][1]['span'][0]['_value']       
        
        try:
            gdate =  gmdt[0]['div'][1]['div'][1]['span'][0]['_value']
            gtime =  gmdt[0]['div'][1]['div'][1]['span'][1]['_value']
        except:
            gdate =  gmdt[0]['div'][1]['div'][0]['span'][0]['_value']
            gtime =  gmdt[0]['div'][1]['div'][0]['span'][1]['_value']
        
        if gdate.lower() == 'завтра':
            gdt = dt(dt.now().year, (dt.now()+ timedelta(days = 1)).month, (dt.now()+ timedelta(days = 1)).day,
                     int(gtime[:2]), int(gtime[-2:]), 0) 
        elif gdate.lower() == 'сегодня':
            gdt = dt(dt.now().year, dt.now().month, dt.now().day,
                     int(gtime[:2]), int(gtime[-2:]), 0)
        else:
            gdt = dt(dt.now().year, MONTH_DICT[gdate[-3:]], int(gdate[:2]), int(gtime[:2]), int(gtime[-2:]), 0)
        
        if dt.today().month > gdt.month:
            gdt = gdt + relativedelta(years = 1)           

        p1 = gmdt[1]['div'][0]['div'][0]['div'][0]['div']
        x  = gmdt[1]['div'][0]['div'][0]['div'][1]['div']
        p2 = gmdt[1]['div'][0]['div'][0]['div'][2]['div']
        coef = {}
        coef[p1[0]['_value']] = float(p1[1]['div'][0]['span'][0]['_value']) #П1
        coef[x[0]['_value']] = float(x[1]['div'][0]['span'][0]['_value']) #X
        coef[p2[0]['_value']] = float(p2[1]['div'][0]['span'][0]['_value']) #П2
        
        return [gdt, OBSDT, team1, team2, COUNTRY, TOURNAMENT, coef['П1'], coef['Х'], coef['П2']]

    def get_games_list(data, OBSDT, COUNTRY, TOURNAMENT, MONTH_DICT):
        ll = []
        for i, elem in enumerate(data):
            try:
                ll.append(sport_page.game_data_extraction(elem, OBSDT, COUNTRY, TOURNAMENT, MONTH_DICT))
            except:
                print(f'ERROR in get_games_list on step {i}')    
        return ll
    
    def cleaning_games(raw_game_list):
        clean_game_list = []
        for elem in raw_game_list:
            try:
                clean_game_list.append(elem['a'][0]['div'])
            except:
                pass
        return clean_game_list
    
    def download_webpage(self, URL):
        self.OBSDT = dt.today().replace(microsecond = 0) #  ВРЕМЯ ИЗМЕРЕНИЯ
        driver = webdriver.Chrome()
        driver.get(URL)
        sleep(10)
        content = driver.page_source
        driver.close()
        self.content_json = html_to_json.convert(content)        
        self.web_data = self.content_json['html'][0]['body'][0]        
        self.one_set_data = self.web_data['div'][0]['div'][1]['div'][0]['div'][1]['div'][2]['div'][0]['div'][0]['div']        
        self.games_row_array = sport_page.cleaning_games(self.one_set_data)        
        
#         self.n_sets = len(self.web_data)
#         self.sets_list = [self.web_data[i]['div'] for i in range(1, self.n_sets)]      
#         self.games_row_array = [[elem[j]['a'][0]['div']  for  j in range(len(elem))]  for elem in self.sets_list]
#         self.games_row_array = list(chain.from_iterable(self.games_row_array))
    
        self.df_games = pd.DataFrame(sport_page.get_games_list(self.games_row_array, self.OBSDT, 
                                                               self.COUNTRY, self.TOURNAMENT, self.MONTH_DICT), 
                        columns = ['GAME_DT', 'OBSDT', 'HT', 'GT', 'COUNTRY', 'TOURNAMENT', 'HW_COEF', 'DR_COEF', 'GW_COEF'])

In [ ]:
page = sport_page(COUNTRY = COUNTRY, TOURNAMENT = TOURNAMENT)
page.download_webpage(URL = URL)

In [ ]:
page.df_games

## Старое

In [ ]:
PAGE_DATA = content_json['html'][0]['body'][0]['div'][0]['div'][1]['div'][0]['div'][2]['div'][1]['div']
one_set_data = PAGE_DATA[2]['div']
one_game_data = one_set_data[0]['a'][0]['div']
game_data_extraction(one_game_data, OBSDT, COUNTRY, TOURNAMENT)
N_SETS = len(PAGE_DATA)
N_SETS
SETS_LIST = [PAGE_DATA[i]['div'] for i in range(1, N_SETS)]
GAMES_ARR = [[elem[j]['a'][0]['div']  for  j in range(len(elem))]  for elem in SETS_LIST]
GAMES_ARR_FLATTEN = list(chain.from_iterable(GAMES_ARR))

In [ ]:
ll = []
for elem in GAMES_ARR_FLATTEN:
    ll.append(game_data_extraction(elem, OBSDT, COUNTRY, TOURNAMENT))

In [ ]:
pd.DataFrame(ll, columns = ['GAME_DT', 'OBSDT', 'HT', 'GT', 'COUNTRY', 'TOURNAMENT', 'HW_COEF', 'DR_COEF', 'GW_COEF'])

In [ ]:
def get_games_df(arr, OBSDT, COUNTRY, TOURNAMENT):
    df = pd.DataFrame(columns = ['GAME_DT', 'OBSDT', 'HT', 'GT', 'COUNTRY', 'TOURNAMENT', 'HW_COEF', 'DR_COEF', 'GW_COEF'])
    for one_game in arr:  
        df.loc[i] = sport_page.game_data_extraction(one_game, OBSDT, COUNTRY, TOURNAMENT)      
    return df

In [ ]:
game_data_extraction(GAMES_ARR_FLATTEN[19], OBSDT, COUNTRY, TOURNAMENT)

In [ ]:
game_data_extraction(aa[19], OBSDT, COUNTRY, TOURNAMENT)

In [ ]:
game_data_extraction(GAMES_ARR[1][4], OBSDT, COUNTRY, TOURNAMENT)

In [ ]:
aa = list(chain.from_iterable(GAMES_ARR))

In [ ]:
GAMES_ARR.shape